## Tablo Veri Setinde Boşlukların Doldurulması Örneği (Imputation)

Gerekli kütüphaneleri yükleyelim.

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

Klasörümüzde bulunan ve Titanic gemisi yolcularına ait veri dosyamızı yükleyelim. Bu geniş veri setinde sadece yolcuların yaşları (Age), ödediği ücretler (Fare) ve kazadan kurtulma durumu (Survived) bizi ilgilendiriyor. O sütunları kullanalım. 

In [2]:
df = pd.read_csv('Titanic.csv',usecols=['Age','Fare','Survived'])
df

,Survived,Age,Fare
0,0,22.0,7.2500
1,1,38.0,71.2833
2,1,26.0,7.9250
3,1,35.0,53.1000
4,0,35.0,8.0500
...,...,...,...
886,0,27.0,13.0000
887,1,19.0,30.0000
888,0,NaN,23.4500
889,1,26.0,30.0000


Oluşan veri çerçevesinde null/NaN veri olup olmadığını sorgulayalım. 

In [3]:
df.isnull().sum()

Survived      0
Age         177
Fare          0
dtype: int64

Age sütununda 177 yolcunun yaş bilgisinin eksik olduğunu görmekteyiz. 
Veri çerçevemiz df'in bir kopyasını alalım ve Age sütununu filtreleyerek, içindeki yaş değerlerinin ortalamasını hesaplayalım. .mean() metodo, NaN verileri önemsemeden ortalamayı hesaplayacaktır.

In [4]:
df_1 = df.copy()
df_1['Age'].mean()

29.69911764705882

#### Strateji 1: NaN verileri düşürmek (Dropping NaN)

İlk strateji, Age sütununda NaN değerlerin olduğu tüm satırları düşürmektir. Bu yöntem kullanışlı gibi görünse de, veri örneğinin çok olmadığı durumlarda ciddi anlamda veri kaybına sebep olabilir. Age sütununda boşlukların olması, diğer sütunlardaki veri değerlerinin kıymetsiz olduğu anlamını beraberinde getirmez. Dolayısıyla, bu stratejinin çok fazla sayıda sütunda aynı anda NaN değer olduğu durumlarda kullanılması daha akıllıcadır.

In [5]:
df_1.dropna(subset=['Age'],how='any',inplace=True)
df_1['Age'].isnull().sum()

0

#### Strateji 2: Boşlukları sabit bir sayı ile doldurmak (Imputation with constant value)
Bu stratejide, ilgilendiğimiz sütundaki boşlukları sabit bir sayı ile dolduracağız. Scikit-learn kütüphanesinin SimpleImputer alt kütüphanesini burada kullanmamız gerekiyor.

In [6]:
from sklearn.impute import SimpleImputer

Veri çerçevemizin yine kopyasını alalım.

In [20]:
df_sabit = df.copy()

Imputer (boşluk doldurucu) fonksiyonumuzu oluşturalım, stratejiyi tanımlayım ve veri setimiz üzerine fit_transform metodu ile uygulayalım.

In [21]:
imputer = SimpleImputer(strategy='constant',fill_value=28)
df_sabit.iloc[:,:] = imputer.fit_transform(df_sabit)
df_sabit.isnull().sum()
df_sabit

,Survived,Age,Fare
0,0.0,22.0,7.2500
1,1.0,38.0,71.2833
2,1.0,26.0,7.9250
3,1.0,35.0,53.1000
4,0.0,35.0,8.0500
...,...,...,...
886,0.0,27.0,13.0000
887,1.0,19.0,30.0000
888,0.0,28.0,23.4500
889,1.0,26.0,30.0000


Görüldüğü üzere, df_sabit veri setindeki Age sütunundaki tüm boşluklar 29 değeri ile doldurulmuştur ve veri setinde boşluklu satır kalmamıştır.

#### Strateji 3: Boşlukları istatistik ile doldurmak
Veri setinde boşluklarını dolduracağımız sütunun mevcut verilerinden elde edilen istatistiki değerler ile de boşluklar doldurulabilir. Bunun için strategy argümanını aşağıdaki şekilde düzenlememiz gerekir.

En sık bulunan değer --> `strategy='most_frequent'`

Ortalama değer       --> `strategy='mean'`

Medyan değer         --> `strategy='median'` 

In [22]:
df_stats = df.copy()

In [23]:
imputer = SimpleImputer(strategy='most_frequent')
df_stats.iloc[:,:] = imputer.fit_transform(df_stats)
df_stats.isnull().sum()
df_stats

,Survived,Age,Fare
0,0.0,22.0,7.2500
1,1.0,38.0,71.2833
2,1.0,26.0,7.9250
3,1.0,35.0,53.1000
4,0.0,35.0,8.0500
...,...,...,...
886,0.0,27.0,13.0000
887,1.0,19.0,30.0000
888,0.0,24.0,23.4500
889,1.0,26.0,30.0000


Aynısını medyan için yapalım.

In [24]:
df_stats2 = df.copy()

In [25]:
imputer = SimpleImputer(strategy='median')
df_stats2.iloc[:,:] = imputer.fit_transform(df_stats2)
df_stats2.isnull().sum()
df_stats2

,Survived,Age,Fare
0,0.0,22.0,7.2500
1,1.0,38.0,71.2833
2,1.0,26.0,7.9250
3,1.0,35.0,53.1000
4,0.0,35.0,8.0500
...,...,...,...
886,0.0,27.0,13.0000
887,1.0,19.0,30.0000
888,0.0,28.0,23.4500
889,1.0,26.0,30.0000


Boşluk doldurma işlemini en yakın komşuluk analizi (k-Nearest Neighbor) kullanarak yapalım. Bunun için scikit-learn kitaplığının impute alt kitaplığındaki KNNImputer fonksiyonunu çağıralım

In [26]:
from sklearn.impute import KNNImputer

In [27]:
df_knn = df.copy()

In [28]:
imputer = KNNImputer(n_neighbors=2, weights="uniform")
df_knn.iloc[:,:] = imputer.fit_transform(df_knn)
df_knn.isnull().sum()
df_knn

,Survived,Age,Fare
0,0.0,22.0,7.2500
1,1.0,38.0,71.2833
2,1.0,26.0,7.9250
3,1.0,35.0,53.1000
4,0.0,35.0,8.0500
...,...,...,...
886,0.0,27.0,13.0000
887,1.0,19.0,30.0000
888,0.0,27.0,23.4500
889,1.0,26.0,30.0000


### Zaman Serilerinde Boşlukların Doldurulması (Time Series Imputation)

In [41]:
city_day = pd.read_csv('city_day.csv',parse_dates=True,index_col='Date')
city_day1=city_day.copy()
city_day.head(20)

,City,PM2.5,PM10,NO,NO2,NOx,NH3,CO,SO2,O3,Benzene,Toluene,Xylene,AQI,AQI_Bucket
Date,,,,,,,,,,,,,,,
2015-01-01,Ahmedabad,NaN,NaN,0.92,18.22,17.15,NaN,0.92,27.64,133.36,0.00,0.02,0.00,NaN,NaN
2015-01-02,Ahmedabad,NaN,NaN,0.97,15.69,16.46,NaN,0.97,24.55,34.06,3.68,5.50,3.77,NaN,NaN
2015-01-03,Ahmedabad,NaN,NaN,17.40,19.30,29.70,NaN,17.40,29.07,30.70,6.80,16.40,2.25,NaN,NaN
2015-01-04,Ahmedabad,NaN,NaN,1.70,18.48,17.97,NaN,1.70,18.59,36.08,4.43,10.14,1.00,NaN,NaN
2015-01-05,Ahmedabad,NaN,NaN,22.10,21.42,37.76,NaN,22.10,39.33,39.31,7.01,18.89,2.78,NaN,NaN
2015-01-06,Ahmedabad,NaN,NaN,45.41,38.48,81.50,NaN,45.41,45.76,46.51,5.42,10.83,1.93,NaN,NaN
2015-01-07,Ahmedabad,NaN,NaN,112.16,40.62,130.77,NaN,112.16,32.28,33.47,0.00,0.00,0.00,NaN,NaN
2015-01-08,Ahmedabad,NaN,NaN,80.87,36.74,96.75,NaN,80.87,38.54,31.89,0.00,0.00,0.00,NaN,NaN
2015-01-09,Ahmedabad,NaN,NaN,29.16,31.00,48.00,NaN,29.16,58.68,25.75,0.00,0.00,0.00,NaN,NaN


In [42]:
# Veri seti içerisindeki eksik verilerin yüzdesel tablosunu hazırlayan fonksiyondur. Doğrudan kullanabilirsiniz.
def missing_values_table(df):
        # Total missing values
        mis_val = df.isnull().sum()
        
        # Percentage of missing values
        mis_val_percent = 100 * df.isnull().sum() / len(df)
        
        # Make a table with the results
        mis_val_table = pd.concat([mis_val, mis_val_percent], axis=1)
        
        # Rename the columns
        mis_val_table_ren_columns = mis_val_table.rename(
        columns = {0 : 'Missing Values', 1 : '% of Total Values'})
        
        # Sort the table by percentage of missing descending
        mis_val_table_ren_columns = mis_val_table_ren_columns[
            mis_val_table_ren_columns.iloc[:,1] != 0].sort_values(
        '% of Total Values', ascending=False).round(1)
        
        # Print some summary information
        print ("Your selected dataframe has " + str(df.shape[1]) + " columns.\n"      
            "There are " + str(mis_val_table_ren_columns.shape[0]) +
              " columns that have missing values.")
        
        # Return the dataframe with missing information
        return mis_val_table_ren_columns

In [43]:
city_day_missing= missing_values_table(city_day)
city_day_missing

Your selected dataframe has 15 columns.
There are 14 columns that have missing values.


,Missing Values,% of Total Values
Xylene,18109,61.3
PM10,11140,37.7
NH3,10328,35.0
Toluene,8041,27.2
Benzene,5623,19.0
AQI,4681,15.9
AQI_Bucket,4681,15.9
PM2.5,4598,15.6
NOx,4185,14.2
O3,4022,13.6


En çok Xylene kirleticisine ait sütunun boşluklar içerdiğini görüyoruz. Bu sütunun ufak bir dilimi ile ilgilenelim.

In [44]:
city_day['Xylene'][50:64]

Date
2015-02-20     7.48
2015-02-21    15.44
2015-02-22     8.47
2015-02-23    28.46
2015-02-24     6.05
2015-02-25     0.81
2015-02-26      NaN
2015-02-27      NaN
2015-02-28      NaN
2015-03-01     1.32
2015-03-02     0.22
2015-03-03     2.25
2015-03-04     1.55
2015-03-05     4.13
Name: Xylene, dtype: float64

Bu küçük dilimde üç adet NaN satırın olduğunu görüyoruz. 

#### Strateji 1: Boşluğu en son gözlemlenen değer ile doldurmak

Bunun için pandas kütüphanesinin ffill methodunu kullanabiliriz. Kullandığımız dilimdeki ilk NaN satırdan hemen önce gözlemlenen Xylene değeri 0.81'dir. Dolayısıyla hemen ardındaki NaN satırın da 0.81 ile dolmasını beklemekteyiz. Diğer NaN satırlar da peşinden geldiği için aynı değer ile dolacaktır.

In [45]:
city_day.fillna(method='ffill',inplace=True)
city_day['Xylene'][50:65]

Date
2015-02-20     7.48
2015-02-21    15.44
2015-02-22     8.47
2015-02-23    28.46
2015-02-24     6.05
2015-02-25     0.81
2015-02-26     0.81
2015-02-27     0.81
2015-02-28     0.81
2015-03-01     1.32
2015-03-02     0.22
2015-03-03     2.25
2015-03-04     1.55
2015-03-05     4.13
2015-03-06     4.13
Name: Xylene, dtype: float64

#### Strateji 2: Boşluğu bir sonraki gözlem değeri ile doldurmak

Bu sefer AQI kirleticisine ait sütundan bir dilim çıkaralım.

In [46]:
city_day['AQI'][20:30]

Date
2015-01-21      NaN
2015-01-22      NaN
2015-01-23      NaN
2015-01-24      NaN
2015-01-25      NaN
2015-01-26      NaN
2015-01-27      NaN
2015-01-28      NaN
2015-01-29    209.0
2015-01-30    328.0
Name: AQI, dtype: float64

Görüleceği üzere, ard arda gelen NaN satırlardan sonra ilk gözlemlenen değer 209.0'dır. Bu değeri, kendisininden önceki NaN değerleri doldurmak için kullanacağız. Bunun için pandas kütüphanesinin bfill yöntemi kullanılır.

In [47]:
city_day.fillna(method='bfill',inplace=True)
city_day['AQI'][20:30]

Date
2015-01-21    209.0
2015-01-22    209.0
2015-01-23    209.0
2015-01-24    209.0
2015-01-25    209.0
2015-01-26    209.0
2015-01-27    209.0
2015-01-28    209.0
2015-01-29    209.0
2015-01-30    328.0
Name: AQI, dtype: float64

#### Strateji 3: Enterpolasyon
Zaman serisi verilerinin zamana karşı birçok varyasyonu vardır. Bu nedenle, geri doldurma (bfill) ve ileri doldurma (ffill) kullanarak hesaplama yapmak, eksik değer sorununu ele almak için en doğru çözüm olmayabilir. Daha uygun bir alternatif, değerlerin artan veya azalan değerlerle doldurulduğu enterpolasyon yöntemlerini kullanmak olacaktır.

Doğrusal (lineer) enterpolasyon, veri noktaları arasında doğrusal bir ilişki olduğunu varsayan ve eksik bir veri noktası için bir değer hesaplamak için bitişik veri noktalarından eksik olmayan değerleri kullanan bir boşluk doldurma tekniğidir.

`pandas.DataFrame.interpolate`fonksiyonunda `methods` argümanı default olarak linear enterpolasyon uygular. Ancak bu argümanın değiştirilmesi ile polinomlar, kubik spline vb. diğer doğrusal olmayan enterpolasyon teknikleri de uygulanabilir.

Aynı fonksiyonda bir diğer opsiyonel argüman ise `limit_direction`olarak karşımıza çıkmaktadır. Bu argüman, enterpolasyonun ileri (baştan sonra), geri (sondan başa) ya da iki yönde de hesaplanmasını sağlamaktadır.

In [48]:
city_day1['Xylene'][50:65]

Date
2015-02-20     7.48
2015-02-21    15.44
2015-02-22     8.47
2015-02-23    28.46
2015-02-24     6.05
2015-02-25     0.81
2015-02-26      NaN
2015-02-27      NaN
2015-02-28      NaN
2015-03-01     1.32
2015-03-02     0.22
2015-03-03     2.25
2015-03-04     1.55
2015-03-05     4.13
2015-03-06      NaN
Name: Xylene, dtype: float64

In [49]:
city_day1.interpolate(method='linear',limit_direction="both",inplace=True)
city_day1['Xylene'][50:65]

Date
2015-02-20     7.4800
2015-02-21    15.4400
2015-02-22     8.4700
2015-02-23    28.4600
2015-02-24     6.0500
2015-02-25     0.8100
2015-02-26     0.9375
2015-02-27     1.0650
2015-02-28     1.1925
2015-03-01     1.3200
2015-03-02     0.2200
2015-03-03     2.2500
2015-03-04     1.5500
2015-03-05     4.1300
2015-03-06     2.2600
Name: Xylene, dtype: float64

### Sıra Sizde

### Egzersiz 1

Elimizde böbrek hastalıkları ile ilgili bir veri seti olsun. Bu veri setinin içindeki kan şekeri (sg) sütununu kullanalım.

In [54]:
df_x =pd.read_csv('kidney_disease.csv', usecols=['sg'])
df_x.head()

,sg
0,1.020
1,1.020
2,1.010
3,1.005
4,1.010


In [55]:
df_x.isnull().sum()

sg    47
dtype: int64

Toplam 47 adet satırda boşluk bulunmaktadır.

Boşlukları medyan ve KNN ile dolduralım.

### Egzersiz 2

`city_day`veri çerçevesinden `NH3` sütununu seçiniz ve en az 100 satırlık bir dilimin `ffill`, `bfill` ve linear enterpolasyon yöntemleri ile boşluklarını doldurunuz.